In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install simpletransformers
!pip install spacy==3.2.4

In [ ]:
import shutil
DATASET_NAME = 'tedtalk2012'
src = '/content/drive/MyDrive/punct-dts.zip'
shutil.copyfile(src, 'punct-dts.zip')
shutil.unpack_archive('punct-dts.zip')
shutil.unpack_archive(f'punct-{DATASET_NAME}.zip')

In [ ]:
import os
import pandas as pd

BASE_DIR = f'./{DATASET_NAME}/'

dataset = {
    
}

for filename in os.listdir(BASE_DIR):
  dataset[filename.replace('.csv', '')] = pd.read_csv(os.path.join(BASE_DIR, filename)).dropna()
 

In [ ]:
shutil.unpack_archive(f'punct-{DATASET_NAME}.zip')

In [ ]:
!pip install wandb

import wandb
wandb.login(key='8e593ae9d0788bae2e0a84d07de0e76f5cf3dcf4')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
from simpletransformers.ner import NERModel, NERArgs
import torch
import wandb



In [ ]:

# Create a new run
project = "bert-base-punct"
# Connect an Artifact to the run
MODEL_NAME = "bert-base-multilingual-cased"
sheet_name = MODEL_NAME.split('-')[2]+'-'+DATASET_NAME
#my_model_artifact = run.use_artifact(my_model_name)

# Download model weights to a folder and return the path
#model_dir = my_model_artifact.download()
train_args = {
    'evaluate_during_training': False,
    'logging_steps': 10,
    'num_train_epochs': 12,
    'save_eval_checkpoints': False,
    'train_batch_size': 16,
    'eval_batch_size': 8,
    'overwrite_output_dir': True,
    'save_eval_checkpoints':False,
    'save_model_every_epoch':False,
    'save_steps':-1,
    'labels_list':dataset['train'].labels.unique().tolist(),
    'use_early_stopping':True,
    'wandb_project':project,
    'wandb_kwargs':{'name': sheet_name}
}


model = NERModel(
    "bert",
    MODEL_NAME,
    args=train_args,
    use_cuda=torch.cuda.is_available()
)
model.train_model(dataset['train'], eval_data=dataset['dev'])
result, model_outputs, wrong_preds = model.eval_model(dataset['test'])

pd.DataFrame.from_dict(result, orient='index').T.to_csv('test_result.csv', index=False,index_label=False)




Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/681M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForTokenClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at 

Downloading:   0%|          | 0.00/972k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

...

Running Epoch 1 of 12:   0%|          | 0/8729 [00:00<?, ?it/s]

Running Epoch 2 of 12:   0%|          | 0/8729 [00:00<?, ?it/s]

Running Epoch 3 of 12:   0%|          | 0/8729 [00:00<?, ?it/s]

Running Epoch 4 of 12:   0%|          | 0/8729 [00:00<?, ?it/s]

Running Epoch 5 of 12:   0%|          | 0/8729 [00:00<?, ?it/s]

In [ ]:
from collections import Counter
import pandas as pd
import re 
from collections import defaultdict
from spacy.matcher import Matcher
from spacy.tokens import Span
from tqdm import tqdm
import spacy

In [ ]:
nlp = spacy.blank('pt')

In [ ]:
def preprocess_data(dataframe):
     

    TOTAL = len(list(dataframe.groupby("sentence_id")))
    data = []
    for _, group in tqdm(dataframe.groupby("sentence_id"), total=TOTAL):
        text = " ".join(group.words)
        doc = nlp.make_doc(text)
        ents = []
        count_dict = defaultdict(lambda : 0)
        for i, label in enumerate(group.labels):
            if label != "O":
                matching_word = group.words.tolist()[i]
                span = Span(doc, i, i+1, label=label)
                ents.append((span.start_char, span.end_char, label.replace('I-','')))
                count_dict[matching_word] += 1
        ent = (text, {
            'entities': ents
        })
        data.append(ent)
    return data

In [ ]:

TEST_DATA = preprocess_data(dataset['test'])

In [ ]:
from spacy.tokens import Doc
from spacy.training import Example
from tqdm.notebook import trange
from zipfile import ZipFile
import numpy as np
import torch

In [ ]:
import shutil


shutil.make_archive(sheet_name, 'zip', './outputs/')
shutil.copyfile(sheet_name+'.zip', f'/content/drive/MyDrive/{sheet_name}.zip')

In [ ]:
from simpletransformers.ner import NERModel, NERArgs
model_args = NERArgs()
model_args.labels_list = ["O", "COMMA", "PERIOD", "QUESTION"]

model = NERModel(
    "bert",
    "./outputs/",
    args=model_args,
    use_cuda=torch.cuda.is_available()
)

In [ ]:
y_true = []
texts = []
for _, group in dataset['test'].groupby("sentence_id"):
        text = " ".join(group.words)
        texts.append(text)
        y_true.append(group.labels.apply(lambda label: label.replace("I-","") ).tolist())

In [ ]:
predictions = model.predict(texts)

In [ ]:

y_pred = []
for i,pred in enumerate(predictions[0], 1):
  y_pred.append(list(map(lambda item:list(item.values())[0].replace("I-",""), pred)))
    

In [ ]:
text = "Eu gostaria de ir para casa agora mas eu não sei como"

In [ ]:
list(predictions[0][0][-1].values())[0]

In [ ]:
from spacy.tokens import Span

predictions_ner = predictions[0]
doc = nlp(texts[0])
doc.set_ents([Span(doc, i, i+1, "I-"+list(item.values())[0]) 
               for i, item in enumerate(predictions_ner[0]) 
                if list(item.values())[0] !="O"])
ents = list(doc.ents)
ents[0].label_

In [ ]:
doc[:2]

In [ ]:
examples = []
for i, (text, entities) in enumerate(TEST_DATA):

    doc = nlp(text)
    doc.set_ents([Span(doc, i, i+1, list(item.values())[0]) 
               for i, item in enumerate(predictions_ner[i]) 
                if list(item.values())[0] !="O"])
  
    example = Example.from_dict(doc, entities)
    examples.append(example)

In [ ]:
example

In [ ]:
entities

In [ ]:
TEST_DATA[:1]

In [ ]:
from spacy.scorer import Scorer, get_ner_prf 
#Compute micro-PRF and per-entity PRF scores.
scores = get_ner_prf(examples)
micro_avg = {
    'f':scores.pop('ents_f'),
    'p':scores.pop('ents_p'),
    'r':scores.pop('ents_r')
} 
scores.update(scores.pop('ents_per_type'))
scores.update({'micro_avg':micro_avg})
scores

In [ ]:
RESULTS_DIR = '/content/drive/MyDrive/BRACIS2022/'
path_to_result = os.path.join(RESULTS_DIR,f'{DATASET_NAME}.xlsx')
if os.path.exists(path_to_result):
  mode = 'a'
else:
  mode = 'w'
mode

In [ ]:

with pd.ExcelWriter(path_to_result, mode=mode) as writer:  
      pd.DataFrame(scores).T.to_excel(writer, sheet_name=sheet_name)